In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from functools import lru_cache
from time import sleep

In [8]:
base_url = 'https://nominatim.openstreetmap.org/search?'

In [9]:
@lru_cache(maxsize=2000) 
def nominatim_geocode(address, format='json', limit=1, **kwargs):

    params = {"q": address, "format": format, "limit":
              limit, **kwargs} 
    response = requests.get(base_url, params=params) 
    response.raise_for_status() 
    
    sleep(1)
    return response.json()

In [38]:
seoul=nominatim_geocode('Seoul')[0]


In [47]:
import folium
map = folium.Map(location=[seoul["lat"], seoul["lon"]])
folium.Marker(location=[seoul["lat"], seoul["lon"]],popup='Seoul').add_to(map)
map

In [50]:
from csv import DictReader,DictWriter
path = 'C:\\Users\\wow\\Desktop'
def read_csv(path):
    with open(path,'r') as f:
        return list(DictReader(f))
def write_csv(data, path,mode='w'):
    if mode not in 'wa':
        raise ValueError("mode should be either 'w' or 'a'")
    with open(path, mode, encoding='UTF-8', newline='') as f:
        writer = DictWriter(f,fieldnames=data[0].keys())
        if mode =='w':
            writer.writeheader()
            
        for row in data:
            writer.writerow(row)

In [51]:
from tqdm import tqdm
def geocode_bulk(data, column='address', verbose=False):
    result, errors = [], []
    for row in tqdm(data):
        try:
            search = nominatim_geocode(row[column],limit=1)
            if len(search)==0:
                result.append(row)
                if verbose:
                    print(f"Cannot find anything for{row[colloum]}")
            else:
                info = search[0]
                info.update(row)
                result.append(info)
        except Exception as e:
            if verbose:
                print(e)
            row['errors'] = e
            errors.append(row)
    if len(errors) > 0 and verbose:
        print(f'{len(errors)}/{len(data)} rows failed')
    return result, errors

In [53]:
path_in = path + '\\cities.csv'
path_out = path + '\\geocoded.csv'
data = read_csv(path_in)
result, errors = geocode_bulk(data, column='name', verbose=True)
result
#write_csv(result, path_out)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


[{'place_id': 199009365,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1543125,
  'boundingbox': ['20.2145811', '35.8984245', '135.8536855', '154.205541'],
  'lat': '35.6828387',
  'lon': '139.7594549',
  'display_name': '東京都, 関東地方, 日本 (Japan)',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.789852179415468,
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png',
  'name': 'Tokyo',
  'population': '38.05',
  'country': 'Japan'},
 {'place_id': 198969764,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 6362934,
  'boundingbox': ['-6.3744575', '-4.9993635', '106.3146732', '106.973975'],
  'lat': '-6.1753942',
  'lon': '106.827183',
  'display_name': 'JKT, Indonesia',
  'class': 'place',
  'type': 'city',
  'importance': 0.657011749371283,
  'icon': 'https://nominatim.o